***
### Import of required libraries
***

In [21]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import pandas as pd

***
### Overall distribution
***

In [22]:
# Import of data
df = pd.read_parquet("/home/jan/STAR_shortcut_OSN_paper/data/EDDM/landing_df.parquet")

# Define the full STAR distances for each STAR
stars = {
    'NAPS1B': 61.9,
    'LAND1B': 74.8,
    'ROKI1A': 52.3,
    'BETO1A': 61.3
}

expected = {
    'NAPS1B': 47,
    'LAND1B': 44,
    'ROKI1A': 43,
    'BETO1A': 53
}

yranges = {
    'NAPS1B': [10, 70],
    'LAND1B': [10, 80],
    'ROKI1A': [10, 60],
    'BETO1A': [10, 70]
}

# Create the subplots with a 1x4 grid (1 row, 4 columns)
fig = make_subplots(
    rows=1, cols=4, 
    shared_xaxes=True,
    horizontal_spacing=0.09,
    subplot_titles=list(stars.keys()),
)

# Loop over each star to generate the subplots
for i, (star, star_distance) in enumerate(stars.items()):
    col = i + 1
    
    # Retrieve the data for the specific star and remove outliers
    temp_df = df.query(f"star == '{star}' and distance < 150 and distance > 10")

    # Calculate the 95th percentile and median for the specific star
    percentile_95 = np.percentile(temp_df.distance, 95)
    median = np.median(temp_df.distance)

    # Add the violin plot for the star
    fig.add_trace(
        go.Violin(
            y=temp_df.distance,
            line_color='#1f77b4',
            meanline_visible=True,
            spanmode='hard',
            name="",
            showlegend=False,
        ),
        row=1, col=col
    )

    # Update the y-axis range
    fig.update_yaxes(
        range=yranges[star],
        col=col,
    )

    # Add the median of the observed distances as a horizontal line (overlay)
    fig.add_shape(
        type="line",
        xref=f"x{col}",
        yref=f"y{col}",
        x0=-0.5, x1=0.5,
        y0=median, y1=median,
        line=dict(color="#2ca02c", width=3, dash="dash"),
        layer="below",
    )

    # Add the 95th percentile as a horizontal line (overlay)
    fig.add_shape(
        type="line",
        xref=f"x{col}",
        yref=f"y{col}",
        x0=-0.5, x1=0.5,
        y0=percentile_95, y1=percentile_95,
        line=dict(color="#ff7f0e", width=3, dash="dash"),
        layer="below",
    )

    # Add the full STAR distance as a horizontal line (overlay)
    fig.add_shape(
        type="line",
        xref=f"x{col}",
        yref=f"y{col}",
        x0=-0.5, x1=0.5,
        y0=star_distance, y1=star_distance,
        line=dict(color="#d62728", width=3, dash="dash"),
        layer="below",
    )

    # Add the expected distance as a horizontal line (overlay)
    fig.add_shape(
        type="line",
        xref=f"x{col}",
        yref=f"y{col}",
        x0=-0.5, x1=0.5,
        y0=expected[star], y1=expected[star],
        line=dict(color="black", width=3, dash="dash"),
        layer="below",
    )

# Add dummy traces for the legend
fig.add_trace(
    go.Scatter(
        x=[None], y=[None],
        mode="lines",
        line=dict(color="#2ca02c", width=3, dash="dash"),
        name="Median of observed distances "
    )
)

fig.add_trace(
    go.Scatter(
        x=[None], y=[None],
        mode="lines",
        line=dict(color="#ff7f0e", width=3, dash="dash"),
        name="95th Percentile of observed distances "
    )
)

fig.add_trace(
    go.Scatter(
        x=[None], y=[None],
        mode="lines",
        line=dict(color="#d62728", width=3, dash="dash"),
        name="Full STAR distance as per procedure "
    )
)

fig.add_trace(
    go.Scatter(
        x=[None], y=[None],
        mode="lines",
        line=dict(color="black", width=3, dash="dash"),
        name="Expected distance as per AIP (off-peak)"
    )
)

# Update layout
fig.update_layout(
    height=700,
    width=2000,
    margin=dict(l=50, r=50, t=100, b=50),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.15,
        xanchor="center",
        x=0.5,
        font=dict(size=25)
    ),
    annotations=[dict(font=dict(size=30), y=1.05)],
)

# Update shared y-axis label
fig.update_yaxes(title_text="Observed Distances [NM]", titlefont=dict(size=25), tickfont=dict(size=25))

# Update x-axis label size
fig.update_xaxes(
    tickfont=dict(size=25)
)

# Show the figure
fig.show()

***
### Distribution Peak / Off-peak
***

#### Add peak / off-peak info

In [23]:
# Extract hour and minute from the "stop" column and create two groups based on time
df['hour'] = pd.to_datetime(df['stop']).dt.hour
df['minute'] = pd.to_datetime(df['stop']).dt.minute

# Create a function to assign the groups based on the specific time ranges
def assign_group(hour, minute):
    time_in_minutes = hour * 60 + minute
    if (
        (time_in_minutes >= 300 and time_in_minutes <= 360) or  # 05:00 - 06:00
        (time_in_minutes >= 420 and time_in_minutes <= 540) or  # 07:00 - 09:00
        (time_in_minutes >= 660 and time_in_minutes <= 840) or  # 11:00 - 14:00
        (time_in_minutes >= 960 and time_in_minutes <= 1200)    # 16:00 - 20:00
    ):
        return 'Peak'
    else:
        return 'Off-peak'

# Apply the function to create the 'group' column
df['group'] = df.apply(lambda row: assign_group(row['hour'], row['minute']), axis=1)

#### Generate plot

In [24]:
# Define the full STAR distances for each STAR
stars = {
    'NAPS1B': 61.9,
    'LAND1B': 74.8,
    'ROKI1A': 52.3,
    'BETO1A': 61.3
}

expected = {
    'NAPS1B': 47,
    'LAND1B': 44,
    'ROKI1A': 43,
    'BETO1A': 53
}

yranges = {
    'NAPS1B': [10, 70],
    'LAND1B': [10, 80],
    'ROKI1A': [10, 60],
    'BETO1A': [10, 70]
}

# Create subplots for each star, setting shared_yaxes=False so each plot has its own Y-axis
fig = make_subplots(rows=1, cols=4, shared_yaxes=False, subplot_titles=list(stars.keys()), horizontal_spacing=0.09)

# Loop over each star to generate the subplots
for i, (star, star_distance) in enumerate(stars.items()):
    col = i + 1
    # Filter data for each star
    temp_df = df.query(f"star == '{star}' and distance < 150 and distance > 10")

    # Create boxplots for each group per star
    fig.add_trace(
        go.Box(
            y=temp_df.query("group == 'Peak'")['distance'],
            name='Peak',
            marker_color='#1f77b4',
            showlegend=False,
            notched=True,
        ), 
        row=1, col=col
    )
    fig.add_trace(
        go.Box(
            y=temp_df.query("group == 'Off-peak'")['distance'],
            name='Off-peak',
            marker_color='#ff7f0e',
            showlegend=False,
            notched=True,
        ), 
        row=1, col=col
    )

    # Add the full STAR distance as a horizontal line (overlay)
    fig.add_shape(
        type="line",
        xref=f"x{col}",
        yref=f"y{col}",
        x0=-0.5, x1=1.5,
        y0=star_distance, y1=star_distance,
        line=dict(color="#d62728", width=3, dash="dash"),
        layer="below",
    )

    # Add the expected distance as a horizontal line (overlay)
    fig.add_shape(
        type="line",
        xref=f"x{col}",
        yref=f"y{col}",
        x0=-0.5, x1=1.5,
        y0=expected[star], y1=expected[star],
        line=dict(color="black", width=3, dash="dash"),
        layer="below",
    )

    # Update the y-axis range for each subplot
    fig.update_yaxes(
        range=yranges[star],
        row=1, col=col,
    )

# Update layout
fig.update_layout(
    height=700,
    width=2000,
    margin=dict(l=50, r=50, t=100, b=50),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.05,
        xanchor="center",
        x=0.5,
        font=dict(size=15)
    ),
    annotations=[dict(font=dict(size=30), y=1.05)]
)

# Update shared y-axis label
fig.update_yaxes(title_text="Observed Distances [NM]", titlefont=dict(size=25), tickfont=dict(size=25))

# Update x-axis label size
fig.update_xaxes(
    tickfont=dict(size=25)
)

# Show plot
fig.show()
